Backup and run loacally at:
https://model.earth/RealityStream/models/location-forest

Load Libraries and data

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Get the raw file contents directly without any HTML wrapping or formatting

df = pd.read_csv("../../input/bees/features/bees-locations.csv")

df = df[['Year', 'State', 'State ANSI', 'Ag District', 'Ag District Code', 'County',
       'County ANSI','Value']]

df = df.pivot(index=['State', 'State ANSI', 'Ag District', 'Ag District Code', 'County',
       'County ANSI',], columns='Year', values='Value').reset_index()

df = df.rename_axis(None, axis=1)

df = df.replace(" (D)", np.NaN)

print(df.head())

     State  State ANSI Ag District  Ag District Code   County  County ANSI  \
0  ALABAMA           1  BLACK BELT                40  AUTAUGA          1.0   
1  ALABAMA           1  BLACK BELT                40  BULLOCK         11.0   
2  ALABAMA           1  BLACK BELT                40   DALLAS         47.0   
3  ALABAMA           1  BLACK BELT                40   ELMORE         51.0   
4  ALABAMA           1  BLACK BELT                40   GREENE         63.0   

  2002 2007 2012 2017 2022  
0  212  201  119  137  258  
1  NaN  NaN  NaN  NaN   48  
2  NaN  NaN   65  209  629  
3  NaN   31  190  151  305  
4  NaN  NaN   14   31   40  


Preprocess the data  
TO DO: Modify to work with all input data, regardless of year

In [14]:
# remove ',' in number
df[2002] = df[2002].str.replace(',', '')
df[2007] = df[2007].str.replace(',', '')
df[2012] = df[2012].str.replace(',', '')
df[2017] = df[2017].str.replace(',', '')
df[2022] = df[2022].str.replace(',', '')
# convert columns to int date type
df[2002] = df[2002].fillna(0).astype(str).astype(int)
df[2007] = df[2007].fillna(0).astype(str).astype(int)
df[2012] = df[2012].fillna(0).astype(str).astype(int)
df[2017] = df[2017].fillna(0).astype(str).astype(int)
df[2022] = df[2022].fillna(0).astype(str).astype(int)
# replace 0 back to nan
df[2002] = df[2002].replace(0, np.NaN)
df[2007] = df[2007].replace(0, np.NaN)
df[2012] = df[2012].replace(0, np.NaN)
df[2017] = df[2017].replace(0, np.NaN)
df[2022] = df[2022].replace(0, np.NaN)
# replace outliers to nan
df[2002] = df[2002].mask(df[2012].sub(df[2012].mean()).div(df[2012].std()).abs().gt(2))
df[2007] = df[2007].mask(df[2012].sub(df[2012].mean()).div(df[2012].std()).abs().gt(2))
df[2012] = df[2012].mask(df[2012].sub(df[2012].mean()).div(df[2012].std()).abs().gt(2))
df[2017] = df[2017].mask(df[2017].sub(df[2017].mean()).div(df[2017].std()).abs().gt(2))
df[2022] = df[2022].mask(df[2022].sub(df[2022].mean()).div(df[2022].std()).abs().gt(2))
# impute nan with mean
df[2002].fillna(value=round(df[2012].mean()), inplace=True)
df[2007].fillna(value=round(df[2012].mean()), inplace=True)
df[2012].fillna(value=round(df[2012].mean()), inplace=True)
df[2017].fillna(value=round(df[2017].mean()), inplace=True)
df[2022].fillna(value=round(df[2022].mean()), inplace=True)



In [15]:
#Checking number of columns in the dataframe
df.head()
df.columns

Index([           'State',       'State ANSI',      'Ag District',
       'Ag District Code',           'County',      'County ANSI',
                     2002,               2007,               2012,
                     2017,               2022],
      dtype='object')

In [16]:
#importing data to calculate Total_Area_Km2, which is useful to calculate bee population density
import requests
import pandas as pd

# API URL
url = "https://api.census.gov/data/2023/geoinfo?get=GEO_ID,NAME,AREALAND,AREAWATR&for=county:*"
# params = {
#     "get": "GEO_ID,NAME,AREALAND,AREAWATR",
#     "for": "county:*"
# }

# Make the API request
response = requests.get(url)

In [17]:
# Check the response status
if response.status_code == 200:
    # Load the response data
    data = response.json()

    # Create a DataFrame from the data
    df_area = pd.DataFrame(data[1:], columns=data[0])

    # Convert AREALAND and AREAWATR to numeric and calculate total area (in square kilometers)
    df_area["AREALAND"] = pd.to_numeric(df_area["AREALAND"])
    df_area["AREAWATR"] = pd.to_numeric(df_area["AREAWATR"])
    df_area["Total_Area_km2"] = (df_area["AREALAND"] + df_area["AREAWATR"]) / 1e6

    # Split NAME into 'county' and 'state' (convert to lowercase)
    df_area[["county", "state"]] = df_area["NAME"].str.split(", ", expand=True)
    df_area["county"] = df_area["county"].str.replace(' County', '', regex=False).str.lower()
    df_area["state"] = df_area["state"].str.lower()

    # Keep only the last 5 digits of GEO_ID as FIPS
    df_area["FIPS"] = df_area["GEO_ID"].str[-5:]

    # Keep only the desired columns
    df_area = df_area[["FIPS", "county", "state", "Total_Area_km2"]]
    
    #df_area.rename(columns={'state': 'State','county':'County'}, inplace=True)

    # Display the resulting DataFrame
    print(df_area)
else:
    print(f"Failed to fetch data. Status Code: {response.status_code}, Response: {response.text}")


       FIPS               county        state  Total_Area_km2
0     01001              autauga      alabama     1565.308997
1     01003              baldwin      alabama     5250.612251
2     01005              barbour      alabama     2342.683364
3     01007                 bibb      alabama     1621.761015
4     01009               blount      alabama     1685.119381
...     ...                  ...          ...             ...
3217  72145  vega baja municipio  puerto rico      176.572672
3218  72147    vieques municipio  puerto rico      683.734210
3219  72149   villalba municipio  puerto rico       95.921207
3220  72151    yabucoa municipio  puerto rico      215.597702
3221  72153      yauco municipio  puerto rico      176.997181

[3222 rows x 4 columns]


In [18]:
# Select only relevant columns for the join
df['County'] = df['County'].str.lower()
df['State']
# Perform an inner join on County name
merged_data = pd.merge(
    df,
    df_area,
    left_on='County',
    right_on='county',
    how='inner'
)

#Create the bee population density column
years = [2002, 2007, 2012, 2017, 2022]

for year in years:
    merged_data[f'{year}_bee_density'] = merged_data[year] / merged_data['Total_Area_km2']
#Earlier code to create bee population increase by year
# df[2007_density] = df[2007] / DF['Total_Area_km2']
#df['2007_increase'] = df.apply(lambda x: 0 if (x[2007] - x[2002]) < 0 else 1, axis=1)
#df['2012_increase'] = df.apply(lambda x: 0 if (x[2012] - x[2007]) < 0 else 1, axis=1)
#df['2017_increase'] = df.apply(lambda x: 0 if (x[2017] - x[2012]) < 0 else 1, axis=1)
#df['2022_increase'] = df.apply(lambda x: 0 if (x[2022] - x[2017]) < 0 else 1, axis=1)
#df['County'] = df['County'].apply(lambda x: x.title()+' County')

print(merged_data)



         State  State ANSI Ag District  Ag District Code    County  \
0      ALABAMA           1  BLACK BELT                40   autauga   
1      ALABAMA           1  BLACK BELT                40   bullock   
2      ALABAMA           1  BLACK BELT                40    dallas   
3      ALABAMA           1  BLACK BELT                40    dallas   
4      ALABAMA           1  BLACK BELT                40    dallas   
...        ...         ...         ...               ...       ...   
14355  WYOMING          56   SOUTHEAST                50    goshen   
14356  WYOMING          56   SOUTHEAST                50   laramie   
14357  WYOMING          56   SOUTHEAST                50  niobrara   
14358  WYOMING          56        WEST                30  sublette   
14359  WYOMING          56        WEST                30     uinta   

       County ANSI   2002   2007   2012   2017   2022   FIPS    county  \
0              1.0  212.0  201.0  119.0  137.0  258.0  01001   autauga   
1          

df_fips is the dataset for national bee population.

In [19]:
states = ["AK", "AL", "AR", "AZ", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "IA",
          "ID", "IL", "IN", "KS", "KY", "LA", "MA", "MD", "ME", "MI", "MN", "MO",
          "MS", "MT", "NC", "ND", "NE", "NH", "NJ", "NM", "NV", "NY", "OH", "OK",
          "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VA", "VT", "WA", "WI",
          "WV", "WY"]
year = 2021
naics_value = 4
df_full = pd.DataFrame()
for state in states:
  url = f"https://raw.githubusercontent.com/ModelEarth/community-timelines/main/training/naics{naics_value}/US/counties/{year}/US-{state}-training-naics{naics_value}-counties-{year}.csv"
  df_ind = pd.read_csv(url)
  df_full = pd.concat([df_full, df_ind], axis=0, join='outer')
df_full['Name'] = df_full['Name'].str.lower()
print(df_full)

     Fips                        Name  Population  Longitude  Latitude  \
0    2013      aleutians east borough         3.0    -162.89     55.05   
1    2016  aleutians west census area         5.0     178.88     51.57   
2    2020      anchorage municipality       293.0    -149.89     61.22   
3    2050          bethel census area        19.0    -162.31     60.39   
4    2060         bristol bay borough         1.0    -156.88     58.74   
..    ...                         ...         ...        ...       ...   
18  56035             sublette county         9.0    -109.99     42.74   
19  56037           sweetwater county        42.0    -108.97     41.62   
20  56039                teton county        23.0    -112.26     47.85   
21  56041                uinta county        21.0    -110.57     41.26   
22  56043             washakie county         8.0    -107.67     43.93   

          Km2  UrbanDensity  PercentUrban  Emp-1133  Est-1133  ...  Pay-3161  \
0    18521.02          0.00    

In [20]:
# Split NAME
df_full['Name'] = df_full['Name'].str.extract(r'^([^ ]+)')


In [21]:
print(df_full)

     Fips        Name  Population  Longitude  Latitude        Km2  \
0    2013   aleutians         3.0    -162.89     55.05   18521.02   
1    2016   aleutians         5.0     178.88     51.57   11746.24   
2    2020   anchorage       293.0    -149.89     61.22    4493.05   
3    2050      bethel        19.0    -162.31     60.39  109939.76   
4    2060     bristol         1.0    -156.88     58.74    1327.31   
..    ...         ...         ...        ...       ...        ...   
18  56035    sublette         9.0    -109.99     42.74   12780.65   
19  56037  sweetwater        42.0    -108.97     41.62   27173.20   
20  56039       teton        23.0    -112.26     47.85   10924.29   
21  56041       uinta        21.0    -110.57     41.26    5409.13   
22  56043    washakie         8.0    -107.67     43.93    5809.13   

    UrbanDensity  PercentUrban  Emp-1133  Est-1133  ...  Pay-3161  Emp-3162  \
0           0.00          0.00       NaN       NaN  ...       NaN       NaN   
1           0

In [22]:
# Convert Fips columns in both dataframes to integers
df_full['Fips'] = df_full['Fips'].astype('Int64')  # Used 'Int64' (capital I) for nullable integers
merged_data['FIPS'] = merged_data['FIPS'].astype('Int64')


In [23]:
#df_fips = df_full[['Name','Fips']].merge(merged_data, how='outer', left_on='Name', right_on='County')

df_fips = df_full[['Name','Fips']].merge(
    merged_data,
    how='outer',
    left_on=['Name', 'Fips'],  
    right_on=['County', 'FIPS']  
)
print(df_fips)

            Name  Fips      State  State ANSI   Ag District  Ag District Code  \
0      aleutians  2013        NaN         NaN           NaN               NaN   
1      aleutians  2016        NaN         NaN           NaN               NaN   
2      anchorage  2020        NaN         NaN           NaN               NaN   
3         bethel  2050        NaN         NaN           NaN               NaN   
4        bristol  2060        NaN         NaN           NaN               NaN   
...          ...   ...        ...         ...           ...               ...   
14696        NaN  <NA>  WISCONSIN        55.0       CENTRAL              50.0   
14697        NaN  <NA>  WISCONSIN        55.0  EAST CENTRAL              60.0   
14698        NaN  <NA>  WISCONSIN        55.0  WEST CENTRAL              40.0   
14699        NaN  <NA>  WISCONSIN        55.0  WEST CENTRAL              40.0   
14700        NaN  <NA>    WYOMING        56.0     NORTHWEST              10.0   

            County  County 

In [24]:
df_fips.to_csv('../../input/bees/targets/bees-targets_new.csv')